In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import gc
import sys
from statsmodels.formula.api import ols
from IPython.display import clear_output
from numba import njit, jit

In [2]:
df = pd.read_csv('DATA_WITH_SALES.csv', sep=';')
df = df[df.Stock > 0]
df.rename(columns={'Stock': 'Demand'}, inplace=True)

df.head(n=5)

,Store_ID,SKU_ID,Date,Sales,Demand,Regular_Price
0,1100,114,01JAN2019,0.0,1.0,51.0
1,1078,114,01JAN2019,0.0,2.0,51.0
2,1191,114,01JAN2019,0.0,2.0,53.0
3,2230,114,01JAN2019,0.0,1.0,40.0
4,1120,114,01JAN2019,0.0,1.0,53.0


In [11]:
popular_pairs = df.groupby(['SKU_ID', 'Store_ID']).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)

In [12]:
popular_pairs.head(n=10)

,SKU_ID,Store_ID,counts
0,98692,1098,90
1,97140,1090,90
2,97140,1087,90
3,97140,1086,90
4,97140,1085,90
5,97140,1084,90
6,97140,1083,90
7,97140,1081,90
8,97128,2834,90
9,97128,2711,90


In [13]:
popular_pairs.head(n=50000).tail(n=1)

,SKU_ID,Store_ID,counts
49999,13156,1086,90


In [42]:
pairs_for_experiment = []
num_of_pairs = 50000
for i, r in popular_pairs.iterrows():
    if i == num_of_pairs:
        break
    pairs_for_experiment.append((r['SKU_ID'], r['Store_ID']))

In [43]:
def transform(x, ttype):
    '''
    Accepts vector and performs transformation on it based on ttype
    '''
    if ttype == 'log':
        return np.log1p(x)
    
    return np.array(x)

In [44]:
def coef(model):
    """
    return regression coefficient of model
    """
    return model.params[1] if len(model.params) > 1 else model.params[0]

def pvalue(model):
    """
    return regression coefficient's t-test p-value of model
    """
    return model.pvalues[1] if len(model.pvalues) > 1 else model.pvalues[0]

In [45]:
def elasticity(X, y, model, model_type):
    '''
    Computes elasticity of model based on X, y, model itself and model's type
    '''
    if model_type == 'lin-lin':
        return coef(model) * np.mean(X) / np.mean(y)
    elif model_type == 'log-lin':
        return coef(model) * np.mean(X)
    elif model_type == 'log-log':
        return coef(model)

In [46]:
def PEDmodel(X, y, model_type='lin-lin'):
    '''
    Makes PED model based on input data X, y and model type
    '''
    _types = model_type.split('-')
    
    act_x = transform(X, _types[1])
    act_x = sm.add_constant(act_x)
    
    act_y = transform(y, _types[0])
    
    return sm.OLS(act_y, act_x).fit()

In [47]:
def get_model_stats(X, y, model, model_type):
    '''
    returns model's statistics
        regression coefficient,
        R-squared,
        T-test p-value,
        elasticity
    '''
    elas = elasticity(X, y, model, model_type)
    
    return coef(model), model.rsquared, pvalue(model), elas

In [49]:
def GeneratePEDModels(dataset):
    model_type = 'log-log'
    sku_id, store_id, elas, qty, ttpvalue = [], [], [], [], []
    iter_count = 0

    for good_id, shop_id in pairs_for_experiment:
        data = dataset[(dataset.Store_ID == shop_id) & (dataset.SKU_ID == good_id)]
        model = PEDmodel(data.Regular_Price, data.Demand, model_type)
        
        c, r2, tp, e = get_model_stats(data.Regular_Price, data.Demand, model, model_type)

        sku_id.append(good_id)
        store_id.append(shop_id)
        elas.append(e)
        ttpvalue.append(tp)
        qty.append(data.Demand.sum() / data.shape[0])
    
    return pd.DataFrame({
        'SKU': sku_id,
        'Store': store_id,
        'Elasticity': elas,
        'Qty': qty,
        'P_value': ttpvalue
    })

In [50]:
trained_data = GeneratePEDModels(df)

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.

In [53]:
trained_data.to_csv('train_data.csv')

In [57]:
df_train = trained_data[(trained_data.Elasticity > -10) & (trained_data.Elasticity < 0) & (trained_data.P_value < 0.05)]

In [62]:
df_train.head()

,SKU,Store,Elasticity,Qty,P_value
3,97140,1086,-2.682751,3.911111,5.429218e-04
4,97140,1085,-5.769920,5.633333,3.125485e-41
7,97140,1081,-6.747417,7.255556,3.004648e-03
8,97128,2834,-2.938277,398.411111,4.927807e-04
11,97128,1954,-8.201317,1.322222,2.705127e-33


In [64]:
product_info = pd.read_excel('PRODUCT_INF.xlsx')
product_info.rename(columns={'Code_AP': 'SKU'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [69]:
product_info[product_info.SKU == 97140]

,ProductName,SKU,ClassId,GroupId,SubGroupId,CategoryId,ClusterId,Class,Groups,SubGroups,Category,Cluster
192058,Контролок_x0020_таб.п.кш.о.20мг_x0020__x2116_14,97140,2003,2047,2099,2532,994,Лекарственные_x0020_и_x0020_профилактические_x...,Пищеварительный_x0020_тракт,Для_x0020_улучшения_x0020_пищеварения,Ингибиторы_x0020_протонной_x0020_помпы,Пантопразол


In [70]:
product_info = product_info[['SKU', 'ClassId', 'CategoryId', 'ClusterId']]

In [71]:
product_info.head()

,SKU,ClassId,CategoryId,ClusterId
0,9999999,-5,-2,-1
1,122139,-5,-2,-1
2,122140,-5,-2,-1
3,122141,-5,-2,-1
4,122142,-5,-2,-1


In [75]:
df_sku = df_train.join(product_info.set_index('SKU'), on='SKU')

In [76]:
df_sku.isna().sum()

SKU           0
Store         0
Elasticity    0
Qty           0
P_value       0
ClassId       0
CategoryId    0
ClusterId     0
dtype: int64

In [77]:
df_sku.head()

,SKU,Store,Elasticity,Qty,P_value,ClassId,CategoryId,ClusterId
3,97140,1086,-2.682751,3.911111,5.429218e-04,2003,2532,994
4,97140,1085,-5.769920,5.633333,3.125485e-41,2003,2532,994
7,97140,1081,-6.747417,7.255556,3.004648e-03,2003,2532,994
8,97128,2834,-2.938277,398.411111,4.927807e-04,2003,2711,39
11,97128,1954,-8.201317,1.322222,2.705127e-33,2003,2711,39


In [92]:
store_info = pd.read_excel('LOCATION_INF.xlsx')
store_info.rename(columns={'TradePointId': 'Store'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [93]:
store_info.head()

,Store,TradePointCode,TradePointName,OpenDate,CloseDate,BrandName,BrandGroupId,Brands_GUID,BE_Id,BE_Name,BE_GUID
0,1076,А01075,СПб_x0020_Новаторов_x0020_б-р_x0020__x0020_8,2009-04-21,2100-01-01,Озерки,3,E378FD87-0BB5-E984-11E8-54EE74216F5D,25,БЕ_x0020_Озерки_x0020_СЗ,_x0035_000DB80-9356-86BC-11E8-82B3A92564D8
1,1077,А01076,СПб_x0020_Малая_x0020_Балканская_x0020_ул.26,2005-06-01,2100-01-01,Озерки,3,E378FD87-0BB5-E984-11E8-54EE74216F5D,25,БЕ_x0020_Озерки_x0020_СЗ,_x0035_000DB80-9356-86BC-11E8-82B3A92564D8
2,1078,А01077,СПб_x0020_Гражданский_x0020_пр._x0020_66_x0020...,2007-11-27,2100-01-01,Озерки,3,E378FD87-0BB5-E984-11E8-54EE74216F5D,25,БЕ_x0020_Озерки_x0020_СЗ,_x0035_000DB80-9356-86BC-11E8-82B3A92564D8
3,1079,А01078,СПб_x0020_Энгельса_x0020_пр._x0020_111_x0020__...,2002-11-04,2100-01-01,Озерки,3,E378FD87-0BB5-E984-11E8-54EE74216F5D,25,БЕ_x0020_Озерки_x0020_СЗ,_x0035_000DB80-9356-86BC-11E8-82B3A92564D8
4,1080,А01079,СПб_x0020_Клочков_x0020_пер._x0020_6,2008-04-16,2100-01-01,Озерки,3,E378FD87-0BB5-E984-11E8-54EE74216F5D,25,БЕ_x0020_Озерки_x0020_СЗ,_x0035_000DB80-9356-86BC-11E8-82B3A92564D8


In [94]:
store_info.BrandGroupId.value_counts()

3    184
Name: BrandGroupId, dtype: int64

In [96]:
store_info.BE_Id.value_counts()

25    184
Name: BE_Id, dtype: int64

In [97]:
store_info.Brands_GUID.value_counts()

E378FD87-0BB5-E984-11E8-54EE74216F5D    184
Name: Brands_GUID, dtype: int64

In [113]:
df_sku.head(n=1)

,SKU,Store,Elasticity,Qty,P_value,ClassId,CategoryId,ClusterId
3,97140,1086,-2.682751,3.911111,0.000543,2003,2532,994


In [114]:
X = df_sku[['SKU', 'Store', 'ClassId', 'CategoryId', 'ClusterId']].to_numpy()
y = df_sku['Elasticity'].to_numpy()

In [115]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [116]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

reg = RandomForestRegressor().fit(X_train, y_train)

In [118]:
mean_squared_error(y_test, reg.predict(X_test))

5.985639072075006